In [2]:
#importing the required packages
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense,Flatten,Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt


In [3]:
#downloading the cat and dogs datasets putting them into directories
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

68608000/68606236 [==============================] - 1s 0us/step


In [4]:
batch_size=32#we are taking 32 images in one batch
image_size=(150,150)# we are resizing the images 
train_data_gen=ImageDataGenerator(
    width_shift_range=0.10,zoom_range=0.10,height_shift_range=0.10,fill_mode="nearest",
    rescale=1/255
)#we are using ImageDatagenerators to augment the data


val_data_gen=ImageDataGenerator(rescale=1/255)

In [5]:
train_data=train_data_gen.flow_from_directory(train_dir,batch_size=batch_size,target_size=image_size,class_mode="categorical")#here we are creating the train and val data
val_data=val_data_gen.flow_from_directory(validation_dir,batch_size=batch_size,target_size=image_size,class_mode="categorical")

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [6]:
conv_base=VGG16(weights="imagenet",include_top=False,input_shape=(150,150,3))#we are going to use Vgg16 and fine tune it
conv_base.trainable=False

58892288/58889256 [==============================] - 1s 0us/step


In [7]:
model=Sequential()
model.add(conv_base)
model.add(Flatten())#we are flattening the output from VGG16
model.add(Dense(64,activation="relu"))#we are creating a dense layer with 64,32 neurons
model.add(Dense(32,activation="relu"))
model.add(Dense(2,activation="softmax"))# here we are taking two output neurons and hence the softmax activation to get the confidence score of both cat and dogs,
                                        #if we use binary classification we wil  get an error regarding output shapes   when deploying to flutter .
model.summary()
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["acc"])#compiling the model with loss as categorical crossentropy and optimizer as adam
es=EarlyStopping(monitor="val_loss")#using early stopping as callback to stop the model training if it starts to overfit

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                524352    
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 66        
Total params: 15,241,186
Trainable params: 526,498
Non-trainable params: 14,714,688
_________________________________________________________________


In [8]:
history=model.fit(train_data,steps_per_epoch=2000//batch_size,epochs=30,validation_data=val_data,validation_steps=1000//batch_size,callbacks=[es])#training the model

Epoch 1/30
62/62 [==============================] - 60s 411ms/step - loss: 0.4892 - acc: 0.7467 - val_loss: 0.3228 - val_acc: 0.8558
Epoch 2/30
62/62 [==============================] - 22s 360ms/step - loss: 0.2649 - acc: 0.8880 - val_loss: 0.2831 - val_acc: 0.8911
Epoch 3/30
62/62 [==============================] - 22s 359ms/step - loss: 0.2106 - acc: 0.8980 - val_loss: 0.2621 - val_acc: 0.8881
Epoch 4/30
62/62 [==============================] - 22s 359ms/step - loss: 0.1807 - acc: 0.9308 - val_loss: 0.4509 - val_acc: 0.8367


In [9]:
conv_base.trainable = True#unfreezing the layers inorder to finetune it
len(conv_base.layers)

19

In [10]:
for layer in conv_base.layers[:17]:#fintuning the model by unfreezing the last two layers
  layer.trainable =  False

In [ ]:
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["acc"])#compiling the model again

In [ ]:
train_dir

'/root/.keras/datasets/cats_and_dogs_filtered/train'

In [11]:
new_model=model.fit(train_data,epochs=30,validation_data=val_data,callbacks=[es],steps_per_epoch=2000//batch_size,validation_steps=1000//batch_size)#training the finetuned model

Epoch 1/30
62/62 [==============================] - 22s 360ms/step - loss: 0.1725 - acc: 0.9350 - val_loss: 0.2930 - val_acc: 0.8851
Epoch 2/30
62/62 [==============================] - 22s 358ms/step - loss: 0.1345 - acc: 0.9497 - val_loss: 0.3147 - val_acc: 0.8851


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
test1=image.load_img('/content/The_Science-Backed_Benefits_of_Being_a_Dog_Owner.jpg',target_size=(150,150,3))
test1=image.img_to_array(test1)
test1=np.expand_dims(test1,axis=0)
model.predict(test1)

array([[1.]], dtype=float32)

In [ ]:
train_data.class_indices

{'cats': 0, 'dogs': 1}

In [ ]:
import tensorflow as tf
keras_file = "cat_dog2.h5"
tf.keras.models.save_model(model , keras_file) #saving the keras model

In [ ]:
test_model=tf.keras.models.load_model("cat_dog2.h5")#loading the saved model

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(test_model)#converting the  saved model to tflite format
tflite_model = converter.convert()
open("cat_dog2.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp7f2bhq5k/assets


INFO:tensorflow:Assets written to: /tmp/tmp7f2bhq5k/assets


60975416